In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## To-do

### Use Plotly, and refresh on pandas.

In [ ]:
def makedf(path):
    df = pd.DataFrame(pd.read_csv(path))
    return df 

In [ ]:
df_oil = makedf('/kaggle/input/store-sales-time-series-forecasting/oil.csv')

In [ ]:
df_oil

In [ ]:
sns.heatmap(df_oil.isnull(),cmap = 'magma',cbar = False);

In [ ]:
df_oil.describe().T

In [ ]:
df1 = df_oil.copy(deep = True)
df1['date'] = pd.to_datetime(df1['date'])
df1.set_index('date',inplace = True)

### Interpolate, fill in missing values based on surrounding values.

In [ ]:
df1['dcoilwtico'].interpolate(method='time', inplace=True)  # Time-based interpolation


#filling in the first NaN value since time-based interpolation relies on behind and front values.
# Backward fill.
df1['dcoilwtico'].fillna(method='bfill', inplace=True) 


In [ ]:
df1

In [ ]:
df1.isna().sum()

In [ ]:
color1 = ['#296C92','#3EB489']
fig,ax = plt.subplots(nrows = 1, ncols = 2,figsize = (15,5))

plt.subplot(1,2,1)
sns.distplot(df1['dcoilwtico']);
plt.title('Distribution : dcoilwtico')

plt.subplot(1,2,2)
sns.distplot(df1['dcoilwtico']);
plt.title('Distribution : dcoilwtico')

fig,ax = plt.subplots(nrows = 1, ncols = 1,figsize = (20,5))
plt.subplot(1,1,1)
sns.lineplot(df1.resample('W').sum());
plt.title('dcoilwtico vs Date')

plt.show()
fig.tight_layout(pad = 2)

In [ ]:
df_train = makedf('/kaggle/input/store-sales-time-series-forecasting/train.csv')
df_train

In [ ]:
df_train.isna().sum()

In [ ]:
df_store = makedf('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
df_store

In [ ]:
df_store.isna().sum()

In [ ]:
df_holy = makedf('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
df_holy

In [ ]:
df_holy.isna().sum()

In [ ]:
df_trans = makedf('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
df_trans

In [ ]:
df_trans.isna().sum()

### df_store into df_train.

In [ ]:
merged_train = pd.merge(df_train, df_store, on='store_nbr', how='left')

In [ ]:
merged_train

In [ ]:
grouped_df = merged_train.groupby('family')

In [ ]:
for name, group in grouped_df:
    print(f"Category: {name}")
    print(group)
    print("\n")
    globals()[f'df_{name.replace(" ", "_").replace("/", "_").replace(",", "_")}'] = pd.DataFrame(group)


### Analysis

graph each store in a category.

graph each category.sum() to compare category relation.

graph states in a category.

In [ ]:
merged_train["family"].unique()

In [ ]:
df_HOME_AND_KITCHEN_II

In [ ]:
familylist = []
familylist = merged_train["family"].unique()

In [ ]:
print(familylist)

In [ ]:
for i in familylist:
    df = globals()[f'df_{i.replace(" ", "_").replace("/", "_").replace(",", "_")}']
    df['date'] = pd.to_datetime(df['date'])
    agg_df = df.groupby('date')['sales'].sum().reset_index()
    globals()[f'agg_{i.replace(" ", "_").replace("/", "_").replace(",", "_")}'] = agg_df

### aggrigated datafram of its date and sales are made. Special characters and spaces are replaced with "_"

In [ ]:
agg_BABY_CARE

In [ ]:


num = len(familylist)
fig, ax = plt.subplots(nrows=num, ncols=1, figsize=(20, 60))

for i, family_member in enumerate(familylist):
    plt.subplot(num, 1, i+1)  # i+1 because subplot indices start from 1
    df = globals()[f'agg_{family_member.replace(" ", "_").replace("/", "_").replace(",", "_")}']
    
    copy_df = df.copy(deep=True)
    
    copy_df['date'] = pd.to_datetime(copy_df['date'])
    copy_df.set_index('date', inplace=True)
    
    df_resam = copy_df.resample('W').sum()
    
    sns.lineplot(x=df_resam.index, y=df_resam['sales'])
    plt.title(f'{family_member} sales')
    
    
fig.tight_layout(pad = 2)
plt.show()


## Time shift to see the patterns in the agg graph.

https://www.kaggle.com/code/thebrownviking20/everything-you-can-do-with-a-time-series/notebook

## Distribution of Categorical Features.

In [ ]:
for i, family_member in enumerate(familylist):
    plt.subplot(num, 1, i+1)  # i+1 because subplot indices start from 1
    df = globals()[f'agg_{family_member.replace(" ", "_").replace("/", "_").replace(",", "_")}']
    

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data['type'] = le.fit_transform(data['type'])